In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key="",
    temperature=0.1,
    max_retries=3,
)

response = llm.invoke("Patient Aarav Mehta, a 29-year-old male, has Type 2 Diabetes. Dietary restrictions include Low sugar, No sweets. Admitted on November 03, 2025 -- suggest food plan")
print(response.content)

E0000 00:00:1762250267.685606   49536 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Of course. Here is a suggested one-day food plan for Aarav Mehta, a 29-year-old male with Type 2 Diabetes, keeping in mind his dietary restrictions of low sugar and no sweets.

This plan is designed to stabilize blood sugar levels, provide sustained energy, and ensure adequate nutrition. It incorporates principles of the diabetic diet, focusing on complex carbohydrates, lean protein, healthy fats, and high fiber, with a cultural consideration for Indian cuisine.

---

### **Dietary Plan for Aarav Mehta**
**Patient:** Aarav Mehta, 29, Male
**Condition:** Type 2 Diabetes
**Admission Date:** November 03, 2025
**Restrictions:** Low Sugar, No Sweets

**Core Principles of this Meal Plan:**
*   **Glycemic Control:** Foods are chosen for their low glycemic index (GI) to prevent sharp spikes in blood sugar.
*   **Portion Control:** Serving sizes are key to managing calorie and carbohydrate intake.
*   **Nutrient Density:** Emphasizes whole foods rich in vitamins, minerals, and fiber.
*   **Regu

In [ ]:
from langchain.prompts import PromptTemplate
Diet_Planing_Prompt = PromptTemplate(input_variables=["Patient_Info"],optional_variables=["Dietian_Notes"],
            template="")



In [18]:

resp = llm.invoke(
    "Patient Aarav Mehta, a 29-year-old male, has Type 2 Diabetes. Dietary restrictions include Low sugar, No sweets. Admitted on November 03, 2025 - suggest a meal plan",
    tools=[GenAITool(google_search={})],
)

print(resp.content)

Okay, Aarav Mehta, 29-year-old male with Type 2 Diabetes, admitted on November 03, 2025. The primary dietary goals are blood sugar control, weight management (if applicable), and adherence to "Low sugar, No sweets."

Here's a suggested meal plan for a typical day in a hospital setting, keeping in mind these restrictions and the need for balanced nutrition. This plan focuses on complex carbohydrates, lean proteins, healthy fats, and plenty of non-starchy vegetables.

---

**Aarav Mehta - Type 2 Diabetes Meal Plan (Low Sugar, No Sweets)**

**General Guidelines:**

*   **Hydration:** Encourage plenty of water throughout the day. Unsweetened tea or coffee is also acceptable. Avoid all sugary drinks (soda, fruit juice, sweetened tea/coffee).
*   **Portion Control:** This is crucial for blood sugar management. Portions listed are general; a hospital dietitian can provide exact measurements.
*   **Cooking Methods:** Prioritize baked, grilled, steamed, or boiled foods. Avoid fried foods.
*   *

PostGreSQL setup


In [2]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="nutriguard",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
print("✅ Connected to NutriGuard database!")
cur.execute("SELECT current_database(), current_user;")
print(cur.fetchall())



✅ Connected to NutriGuard database!
[('nutriguard', 'postgres')]


In [20]:
import psycopg2
from datetime import datetime

# Connect to DB
conn = psycopg2.connect(
    dbname="nutriguard",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

patient_id = 1
cur.execute("SET search_path TO nutriguard;")

# Fetch patient details
cur.execute("SELECT * FROM patients WHERE id = %s;", (patient_id,))
patient = cur.fetchone()

# Fetch nutrition entries for that patient
cur.execute("SELECT * FROM nutrition_entries WHERE patient_id = %s;", (patient_id,))
entries = cur.fetchall()

# --- Function for patient details ---
def record_to_sentence(record):
    (
        id, name, age, gender,
        medical_conditions,
        dietary_restrictions,
        admission_date,
        photo_url,
        created_at,
        updated_at
    ) = record

    return (
        f"Patient {name}, a {age}-year-old {gender.lower()}, "
        f"diagnosed with {medical_conditions if medical_conditions else 'no significant medical conditions'}. "
        f"Dietary restrictions: {dietary_restrictions if dietary_restrictions else 'none'}. "
        f"Admitted on {admission_date.strftime('%B %d, %Y')}."
    )

# --- Function for nutrition record ---
def nutrition_to_sentence(record):
    (
        id,
        patient_id,
        recipe_id,
        recorded_by,
        date,
        intake_percentage,
        blood_sugar,
        notes,
        created_at
    ) = record

    sentence = (
        f"On {date.strftime('%B %d, %Y')}, the patient had a meal with an intake of {intake_percentage}% "
        f"and blood sugar level of {blood_sugar} mg/dL. "
        f"{'Notes: ' + notes if notes else 'No additional notes.'}"
    )
    return sentence

# --- Generate the combined LLM-ready summary ---
if patient:
    summary = record_to_sentence(patient)
    if entries:
        summary += " Nutrition tracking since admission includes: "
        for e in entries:
            summary += " " + nutrition_to_sentence(e)
    else:
        summary += " No nutrition records found yet."

    print("🧠 LLM Prompt:\n")
    print(summary)
else:
    print("❌ Patient not found.")


🧠 LLM Prompt:

Patient Aarav Mehta, a 29-year-old male, diagnosed with Type 2 Diabetes. Dietary restrictions: Low sugar, No sweets. Admitted on November 03, 2025. Nutrition tracking since admission includes:  On November 01, 2025, the patient had a meal with an intake of 95.00% and blood sugar level of 128.50 mg/dL. Notes: Patient ate well, blood sugar within target range On November 01, 2025, the patient had a meal with an intake of 100.00% and blood sugar level of 142.00 mg/dL. Notes: Completed lunch, slight elevation in blood sugar On November 01, 2025, the patient had a meal with an intake of 85.00% and blood sugar level of 135.00 mg/dL. Notes: Left some vegetables, blood sugar acceptable


In [21]:
summary

'Patient Aarav Mehta, a 29-year-old male, diagnosed with Type 2 Diabetes. Dietary restrictions: Low sugar, No sweets. Admitted on November 03, 2025. Nutrition tracking since admission includes:  On November 01, 2025, the patient had a meal with an intake of 95.00% and blood sugar level of 128.50 mg/dL. Notes: Patient ate well, blood sugar within target range On November 01, 2025, the patient had a meal with an intake of 100.00% and blood sugar level of 142.00 mg/dL. Notes: Completed lunch, slight elevation in blood sugar On November 01, 2025, the patient had a meal with an intake of 85.00% and blood sugar level of 135.00 mg/dL. Notes: Left some vegetables, blood sugar acceptable'

In [30]:
# 1️⃣ Rollback the failed transaction
conn.rollback()

# 2️⃣ Then run your query again
cur.execute("SELECT name, description FROM nutriguard.recipes ORDER BY name;")
recipes = cur.fetchall()

if recipes:
    recipe_text = ", ".join(
        [f"{i+1}. {name} – {desc if desc else 'No description'}" for i, (name, desc) in enumerate(recipes)]
    )
    print(recipe_text)
else:
    print("No recipes found.")


1. Diabetic-Friendly Salmon Bowl – Baked salmon with quinoa and spinach, low glycemic index, 2. Gluten-Free Oatmeal Breakfast – Steel-cut oats with blueberries and low-fat milk, 3. Grilled Chicken with Steamed Vegetables – Lean grilled chicken breast served with steamed broccoli and carrots, 4. High-Protein Greek Yogurt Parfait – Greek yogurt layered with blueberries (nut-free), 5. Kidney-Friendly Lentil Soup – Low potassium lentil soup with carrots and herbs, 6. Low-Carb Tofu Scramble – Scrambled tofu with spinach and tomatoes, 7. Low-Sodium Turkey Stir-Fry – Turkey breast with mixed vegetables in olive oil, 8. Mediterranean Quinoa Salad – Quinoa with tomatoes, spinach, and olive oil dressing, 9. Pureed Sweet Potato Blend – Smooth pureed sweet potato for dysphagia patients, 10. Soft Diet Chicken Puree – Finely pureed chicken with gravy for post-surgical patients


In [10]:
cur.close()
conn.close()